In [1]:
#------------------------------------------------------------------------------
# This script receives video from the HoloLens front RGB camera and plays it.
# The camera supports various resolutions and framerates. See
# https://github.com/jdibenes/hl2ss/blob/main/etc/pv_configurations.txt
# for a list of supported formats. The default configuration is 1080p 30 FPS. 
# The stream supports three operating modes: 0) video, 1) video + camera pose, 
# 2) query calibration (single transfer).
# Press esc to stop.
#------------------------------------------------------------------------------

from pynput import keyboard

import cv2
import hl2ss_imshow
import hl2ss
import hl2ss_lnm

# Settings --------------------------------------------------------------------

# HoloLens address
host = "192.168.0.114"

# Operating mode
# 0: video
# 1: video + camera pose
# 2: query calibration (single transfer)
mode = hl2ss.StreamMode.MODE_0

# Enable Mixed Reality Capture (Holograms)
enable_mrc = False

# Camera parameters
width     = 320
height    = 288
framerate = 5

# Framerate denominator (must be > 0)
# Effective FPS is framerate / divisor
divisor = 1 

# Video encoding profile
profile = hl2ss.VideoProfile.H265_MAIN

# Decoded format
# Options include:
# 'bgr24'
# 'rgb24'
# 'bgra'
# 'rgba'
# 'gray8'
decoded_format = 'bgr24'

#------------------------------------------------------------------------------

hl2ss_lnm.start_subsystem_pv(host, hl2ss.StreamPort.PERSONAL_VIDEO, enable_mrc=enable_mrc)

if (mode == hl2ss.StreamMode.MODE_2):
    data = hl2ss_lnm.download_calibration_pv(host, hl2ss.StreamPort.PERSONAL_VIDEO, width, height, framerate)
    print('Calibration')
    print(f'Focal length: {data.focal_length}')
    print(f'Principal point: {data.principal_point}')
    print(f'Radial distortion: {data.radial_distortion}')
    print(f'Tangential distortion: {data.tangential_distortion}')
    print('Projection')
    print(data.projection)
    print('Intrinsics')
    print(data.intrinsics)
else:
    enable = True

    def on_press(key):
        global enable
        enable = key != keyboard.Key.esc
        return enable

    listener = keyboard.Listener(on_press=on_press)
    listener.start()

    client = hl2ss_lnm.rx_pv(host, hl2ss.StreamPort.PERSONAL_VIDEO, mode=mode, width=width, height=height, framerate=framerate, divisor=divisor, profile=profile, decoded_format=decoded_format)
    client.open()

    while (enable):
        data = client.get_next_packet()

        print(f'Pose at time {data.timestamp}')
        print(data.pose)
        print(f'Focal length: {data.payload.focal_length}')
        print(f'Principal point: {data.payload.principal_point}')

        cv2.imshow('Video', data.payload.image)
        cv2.waitKey(1)

    client.close()
    listener.join()

hl2ss_lnm.stop_subsystem_pv(host, hl2ss.StreamPort.PERSONAL_VIDEO)


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
#------------------------------------------------------------------------------
# This script receives video from the HoloLens depth camera in ahat mode and 
# plays it. The resolution is 512x512 @ 45 FPS. The stream supports three 
# operating modes: 0) video, 1) video + rig pose, 2) query calibration (single 
# transfer). Depth and AB data are scaled for visibility. The ahat and long 
# throw streams cannot be used simultaneously.
# Press esc to stop.
# See https://github.com/jdibenes/hl2ss/tree/main/extensions before setting
# profile_z to hl2ss.DepthProfile.ZDEPTH (lossless* compression).
#------------------------------------------------------------------------------

from pynput import keyboard

import numpy as np
import cv2
import hl2ss_imshow
import hl2ss
import hl2ss_lnm
import open3d as o3d

# Settings --------------------------------------------------------------------

# HoloLens address
host = "192.168.0.114"

# Operating mode
# 0: video
# 1: video + rig pose
# 2: query calibration (single transfer)
mode = hl2ss.StreamMode.MODE_1

# Framerate denominator (must be > 0)
# Effective framerate is framerate / divisor
divisor = 1 

# Depth encoding profile
profile_z = hl2ss.DepthProfile.SAME

# Video encoding profile
profile_ab = hl2ss.VideoProfile.H265_MAIN

# Initialize Open3D Visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()

# Create a point cloud object
point_cloud = o3d.geometry.PointCloud()

#------------------------------------------------------------------------------

if (mode == hl2ss.StreamMode.MODE_2):
    data = hl2ss_lnm.download_calibration_rm_depth_ahat(host, hl2ss.StreamPort.RM_DEPTH_AHAT)
    print('Calibration data')
    print('Image point to unit plane')
    print(data.uv2xy)
    print('Extrinsics')
    print(data.extrinsics)
    print(f'Scale: {data.scale}')
    print(f'Alias: {data.alias}')
    print('Undistort map')
    print(data.undistort_map)
    print('Intrinsics (undistorted only)')
    print(data.intrinsics)
    quit()

enable = True

def on_press(key):
    global enable
    enable = key != keyboard.Key.esc
    return enable

listener = keyboard.Listener(on_press=on_press)
listener.start()

client = hl2ss_lnm.rx_rm_depth_ahat(host, hl2ss.StreamPort.RM_DEPTH_AHAT, mode=mode, divisor=divisor, profile_z=profile_z, profile_ab=profile_ab)
client.open()

while (enable):
    data = client.get_next_packet()
    
    print(f'Pose at time {data.timestamp}')
    print(data.pose)
    
    height, width = data.payload.depth.shape
    y, x = np.indices((height, width))
    y=-y
    z = data.payload.depth / 1000.0  # Convert depth values from millimeters to meters

    # Create a numpy array with point cloud data
    points = np.column_stack((x.reshape(-1), y.reshape(-1), z.reshape(-1)))

    # Update the Open3D point cloud with colors
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Clear and update the Open3D visualizer
    vis.clear_geometries()
    vis.add_geometry(point_cloud)
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()

    cv2.imshow('Depth', data.payload.depth / np.max(data.payload.depth)) # Scaled for visibility
    cv2.imshow('AB', data.payload.ab / np.max(data.payload.ab)) # Scaled for visibility
    cv2.waitKey(1)

client.close()
listener.join()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Pose at time 8772936263
[[ 0.50473416 -0.8593098  -0.08264488  0.        ]
 [-0.25457373 -0.05668211 -0.9653908   0.        ]
 [ 0.82488525  0.50830483 -0.24736714  0.        ]
 [-0.6406706   0.04802474 -0.37731904  1.        ]]
Pose at time 8773154850
[[ 0.5048421  -0.8592392  -0.08271825  0.        ]
 [-0.2547263  -0.05672991 -0.9653476   0.        ]
 [ 0.824772    0.5084186  -0.24751043  0.        ]
 [-0.640748    0.04805484 -0.37735945  1.        ]]
Pose at time 8773374856
[[ 0.50499    -0.85914654 -0.08277944  0.        ]
 [-0.2549333  -0.05683899 -0.9652868   0.        ]
 [ 0.8246176   0.50856346 -0.24772823  0.        ]
 [-0.64081794  0.04809192 -0.37741807  1.        ]]
Pose at time 8773594860
[[ 0.50523376 -0.8589926  -0.08288851  0.        ]
 [-0.25521994 -0.05697525 -0.96520275  0.        ]
 [ 0.82437944  0.5

Pose at time 8780936430
[[ 0.5065848  -0.85811234 -0.08375466  0.        ]
 [-0.25573882 -0.05677986 -0.96507686  0.        ]
 [ 0.82338893  0.5103125  -0.24821639  0.        ]
 [-0.6415252   0.04850566 -0.37733594  1.        ]]
Pose at time 8781154994
[[ 0.50678086 -0.8579933  -0.08378863  0.        ]
 [-0.25576037 -0.05682158 -0.96506876  0.        ]
 [ 0.8232616   0.5105081  -0.24823666  0.        ]
 [-0.6415935   0.04853025 -0.37733454  1.        ]]
Pose at time 8781374998
[[ 0.5069872  -0.85787064 -0.08379731  0.        ]
 [-0.2557556  -0.05687904 -0.96506673  0.        ]
 [ 0.8231361   0.51070815 -0.24824214  0.        ]
 [-0.64163923  0.04858178 -0.37736     1.        ]]
Pose at time 8781595004
[[ 0.5072274  -0.85772634 -0.08382139  0.        ]
 [-0.25574005 -0.05692399 -0.96506846  0.        ]
 [ 0.82299304  0.5109458  -0.24822855  0.        ]
 [-0.64168483  0.04863331 -0.3773579   1.        ]]
Pose at time 8781815006
[[ 0.50752723 -0.85754263 -0.08388561  0.        ]
 [-0.2557

Pose at time 8802694556
[[ 0.5172721  -0.8507812  -0.09274036  0.        ]
 [-0.24420416 -0.04287267 -0.9687755   0.        ]
 [ 0.8202401   0.52376807 -0.22994113  0.        ]
 [-0.64379466  0.05193967 -0.37153637  1.        ]]
Pose at time 8802935920
[[ 0.517406   -0.8507035  -0.09270692  0.        ]
 [-0.24435213 -0.04304814 -0.96873045  0.        ]
 [ 0.8201116   0.5238801  -0.2301445   0.        ]
 [-0.6440247   0.05197057 -0.3716486   1.        ]]
Pose at time 8803154525
[[ 0.5176666  -0.8505416  -0.09273875  0.        ]
 [-0.24458933 -0.04324698 -0.968662    0.        ]
 [ 0.81987655  0.524127   -0.23042107  0.        ]
 [-0.644241    0.05203369 -0.37175846  1.        ]]
Pose at time 8803374531
[[ 0.51813865 -0.8502399  -0.09286821  0.        ]
 [-0.24492314 -0.04346406 -0.9685677   0.        ]
 [ 0.81947845  0.5245979  -0.2307638   0.        ]
 [-0.6445217   0.05211142 -0.3719051   1.        ]]
Pose at time 8803594532
[[ 0.5187427  -0.8498455  -0.09310561  0.        ]
 [-0.2453

Pose at time 8825594954
[[ 0.5180406  -0.8513037  -0.08316189  0.        ]
 [-0.24114525 -0.05207503 -0.96909094  0.        ]
 [ 0.82066005  0.52208257 -0.23226488  0.        ]
 [-0.6436469   0.05088601 -0.37248385  1.        ]]
Pose at time 8825814957
[[ 0.51845443 -0.85107124 -0.08296171  0.        ]
 [-0.2407346  -0.05217469 -0.9691875   0.        ]
 [ 0.8205192   0.5224513  -0.23193252  0.        ]
 [-0.64366883  0.05094492 -0.3724      1.        ]]
Pose at time 8826034965
[[ 0.51898956 -0.85076004 -0.08280891  0.        ]
 [-0.24030322 -0.05224621 -0.9692909   0.        ]
 [ 0.82030743  0.5229512  -0.2315557   0.        ]
 [-0.64369714  0.05101493 -0.37234658  1.        ]]
Pose at time 8826254965
[[ 0.5194188  -0.85053027 -0.08247691  0.        ]
 [-0.23988736 -0.05249727 -0.9693803   0.        ]
 [ 0.8201574   0.5232995  -0.23129952  0.        ]
 [-0.6436761   0.05106881 -0.3723251   1.        ]]
Pose at time 8826474969
[[ 0.5196442  -0.85043013 -0.08208942  0.        ]
 [-0.2395

Pose at time 8843156077
[[ 0.03880787 -0.9684975   0.24598089  0.        ]
 [-0.31826732 -0.2453264  -0.91570795  0.        ]
 [ 0.9472065  -0.04275098 -0.31776166  0.        ]
 [-0.6150776  -0.04631692 -0.44811946  1.        ]]
Pose at time 8843376082
[[-0.01094723 -0.9649465   0.26221743  0.        ]
 [-0.31952393 -0.24510884 -0.91532815  0.        ]
 [ 0.9475148  -0.09380531 -0.3056401   0.        ]
 [-0.61575174 -0.04776984 -0.4479904   1.        ]]
Pose at time 8843596088
[[-0.05574048 -0.9595495   0.27596688  0.        ]
 [-0.32044905 -0.24458146 -0.91514605  0.        ]
 [ 0.94562435 -0.13944405 -0.29385376  0.        ]
 [-0.61648184 -0.04876305 -0.44807053  1.        ]]
Pose at time 8843816090
[[-0.09120417 -0.95377386  0.28635117  0.        ]
 [-0.32197365 -0.24386227 -0.9148027   0.        ]
 [ 0.94234526 -0.17563137 -0.2848488   0.        ]
 [-0.6170577  -0.0503931  -0.44790477  1.        ]]
Pose at time 8844036096
[[-0.12351036 -0.9467062   0.29747736  0.        ]
 [-0.3253

...

Pose at time 8916476450
[[-0.35480452 -0.8927641   0.2776436   0.        ]
 [-0.29622427 -0.17431998 -0.9390759   0.        ]
 [ 0.88677216 -0.4154331  -0.20260882  0.        ]
 [-0.6097669  -0.0455029  -0.42549837  1.        ]]
Pose at time 8916696453
[[-0.33058774 -0.9043077   0.2700734   0.        ]
 [-0.2949411  -0.17283797 -0.9397538   0.        ]
 [ 0.89650536 -0.39032665 -0.20957959  0.        ]
 [-0.6089531  -0.04414721 -0.42600003  1.        ]]
Pose at time 8916937848
[[-0.29322493 -0.92018455  0.25938302  0.        ]
 [-0.2961939  -0.17052245 -0.93978256  0.        ]
 [ 0.90900403 -0.35239536 -0.22255158  0.        ]
 [-0.60822344 -0.04269013 -0.42661476  1.        ]]
Pose at time 8917156460
[[-0.24627829 -0.9370315   0.24762574  0.        ]
 [-0.2991191  -0.16953778 -0.93903375  0.        ]
 [ 0.9218863  -0.3053334  -0.2385304   0.        ]
 [-0.6076711  -0.04115793 -0.4273122   1.        ]]
Pose at time 8917376461
[[-0.19226432 -0.9530592   0.23390645  0.        ]
 [-0.3017

Pose at time 8943597054
[[ 0.5490277  -0.8354593  -0.02400613  0.        ]
 [-0.3636418  -0.21291113 -0.9068811   0.        ]
 [ 0.75255114  0.50663245 -0.42070198  0.        ]
 [-0.37676787 -0.00141203 -0.3753772   1.        ]]
Pose at time 8944257066
[[ 0.54884404 -0.8355842  -0.02385572  0.        ]
 [-0.36417145 -0.2133193  -0.9065724   0.        ]
 [ 0.7524289   0.5062544  -0.42137492  0.        ]
 [-0.37686238 -0.00155023 -0.37593472  1.        ]]
Pose at time 8944948817
[[ 0.54814225 -0.8360595  -0.02333391  0.        ]
 [-0.36410934 -0.21341753 -0.9065745   0.        ]
 [ 0.7529704   0.50542784 -0.4214003   0.        ]
 [-0.37663347 -0.00197142 -0.37627578  1.        ]]
Pose at time 8945597073
[[ 0.5467609  -0.8370176  -0.02130762  0.        ]
 [-0.36474484 -0.21519935 -0.9058975   0.        ]
 [ 0.75366676  0.5030812  -0.42296052  0.        ]
 [-0.37628528 -0.00283873 -0.376711    1.        ]]
Pose at time 8946257084
[[ 0.54548025 -0.83790076 -0.01933673  0.        ]
 [-0.3662

Pose at time 8968949125
[[ 0.5385152  -0.8425843  -0.007294    0.        ]
 [-0.37383527 -0.23115122 -0.8982297   0.        ]
 [ 0.7551482   0.48643717 -0.43946624  0.        ]
 [-0.3779984  -0.00768274 -0.38182786  1.        ]]
Pose at time 8969597389
[[ 0.5385351  -0.8425653  -0.00797468  0.        ]
 [-0.37462926 -0.23094952 -0.89795053  0.        ]
 [ 0.75474024  0.48656538 -0.44002402  0.        ]
 [-0.37750843 -0.00741315 -0.3805159   1.        ]]
Pose at time 8970257400
[[ 0.538586   -0.8425323  -0.0080128   0.        ]
 [-0.37458453 -0.2309115  -0.89797866  0.        ]
 [ 0.75472605  0.4866401  -0.43996525  0.        ]
 [-0.37762564 -0.00737837 -0.38032696  1.        ]]
Pose at time 8970939702
[[ 0.538668   -0.84248006 -0.00800499  0.        ]
 [-0.3743199  -0.23080027 -0.89811784  0.        ]
 [ 0.7547989   0.48678368 -0.43968177  0.        ]
 [-0.37771773 -0.00731125 -0.3801821   1.        ]]
Pose at time 8971598326
[[ 0.539003   -0.8422675  -0.00781661  0.        ]
 [-0.3744

Pose at time 8983598597
[[ 0.5381602  -0.8428191  -0.00628713  0.        ]
 [-0.37088954 -0.23011017 -0.8997166   0.        ]
 [ 0.7568517   0.48652348 -0.4364289   0.        ]
 [-0.37786743 -0.00683007 -0.3781377   1.        ]]
Pose at time 8983818599
[[ 0.53845716 -0.8426286  -0.00639978  0.        ]
 [-0.3708533  -0.23014975 -0.89972174  0.        ]
 [ 0.7566583   0.48683503 -0.43641734  0.        ]
 [-0.37789446 -0.00676349 -0.3781504   1.        ]]
Pose at time 8984038603
[[ 0.53868306 -0.8424839  -0.00643468  0.        ]
 [-0.3708259  -0.23023379 -0.8997114   0.        ]
 [ 0.75651085  0.4870454  -0.43643785  0.        ]
 [-0.377917   -0.00670624 -0.37815696  1.        ]]
Pose at time 8984258607
[[ 0.5388564  -0.84237313 -0.00642535  0.        ]
 [-0.37090823 -0.23040366 -0.89963406  0.        ]
 [ 0.75634706  0.4871568  -0.4365977   0.        ]
 [-0.3779717  -0.00668341 -0.37814826  1.        ]]
Pose at time 8984478610
[[ 0.5389645  -0.8423035  -0.00649017  0.        ]
 [-0.3710

Pose at time 8993778772
[[ 0.5369371  -0.8436134  -0.00385839  0.        ]
 [-0.3673079  -0.22965932 -0.90129983  0.        ]
 [ 0.7594626   0.48535857 -0.43317854  0.        ]
 [-0.37579358 -0.00761631 -0.37634394  1.        ]]
Pose at time 8994438783
[[ 0.5370769  -0.8435263  -0.00343356  0.        ]
 [-0.36732453 -0.230209   -0.90115285  0.        ]
 [ 0.75935566  0.48524964 -0.4334879   0.        ]
 [-0.37588117 -0.00762251 -0.37643233  1.        ]]
Pose at time 8995098794
[[ 0.5372965  -0.8433872  -0.00325632  0.        ]
 [-0.36775964 -0.23081064 -0.90082157  0.        ]
 [ 0.7589897   0.4852058  -0.43417752  0.        ]
 [-0.37605014 -0.00763074 -0.37656978  1.        ]]
Pose at time 8995758806
[[ 0.5376095  -0.84318626 -0.00358713  0.        ]
 [-0.36824593 -0.23095977 -0.90058464  0.        ]
 [ 0.7585321   0.48548383 -0.43466616  0.        ]
 [-0.3761796  -0.00755897 -0.37672105  1.        ]]
Pose at time 8996418817
[[ 0.537977   -0.8429503  -0.00391099  0.        ]
 [-0.3685